In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import os
import csv
import pandas as pd
import PoseClassModule as ps

In [ ]:
from sklearn.preprocessing import StandardScaler 
from sklearn.ensemble import RandomForestClassifier
import pickle 
from sklearn.model_selection import train_test_split

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [ ]:
df = pd.read_csv('coords.csv')

In [ ]:
X = df.drop('class', axis=1) 
y = df['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)


In [ ]:
rf = RandomForestClassifier()
model = rf.fit(X_train, y_train)

In [ ]:
#Saving model
with open('class_model.pkl', 'wb') as f:
    pickle.dump(model, f)

with open('class_model.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
#Detecting pose class
cap = cv2.VideoCapture(os.path.join(ps.file_path, 'posing_2x.mp4'))

with mp_pose.Pose(min_detection_confidence=0.4, min_tracking_confidence=0.4) as pose:

    while cap.isOpened():

        ret, frame = cap.read()
        if ret==False: #break if video ended
            break

        frame=cv2.resize(frame, (frame.shape[1] // 2,frame.shape[0] // 2))
        # Recolor to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Estimation
        results = pose.process(image)
    
        # Recolor to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                   
        # Drawing
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)         

        try:
            # Extract Pose landmarks
            pose_class = results.pose_landmarks.landmark
            row= []
            for landmark in range(len(pose_class)):
                row.extend([pose_class[landmark].x, pose_class[landmark].y, pose_class[landmark].z, pose_class[landmark].visibility])
                
            X = pd.DataFrame([row])
            pose_class = model.predict(X)[0]
            pose_prob = model.predict_proba(X)[0]

            for i in range(5):
                
                if pose_prob[i] > 0.75:
                   
                    cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
                    
                    # Pose class
                    cv2.putText(image, 'POSE'
                                , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                    cv2.putText(image, pose_class.split(' ')[0]
                                , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                    
                    # Probability
                    cv2.putText(image, 'SCORE'
                                , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                    cv2.putText(image, str(round(pose_prob[np.argmax(pose_prob)],2))
                                , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
     
        except:
            pass      
        
        cv2.imshow('Posing', image)

        k = cv2.waitKey(1) & 0xFF
        if k == 27:         #ESC key to exit
            break

cap.release()
cv2.destroyAllWindows()